# Mage Wavelengths

In [35]:
# imports
import os
import numpy as np
from pkg_resources import resource_filename

from scipy.io import readsav 

from astropy.io import fits
from astropy.table import Table

from pypeit.core.wavecal import templates

from linetools import utils as ltu

## Load up a few files

In [3]:
mase_path = os.path.join(os.getenv('XIDL_DIR'), 'Magellan', 'MAGE', 'mase', 'Calib')

### FITS

In [4]:
fits_file = os.path.join(mase_path, 'mage_archive_orders.fits')

In [6]:
hdu = fits.open(fits_file)
hdu.info()

Filename: /u/xavier/local/idl/xidl/Magellan/MAGE/mase/Calib/mage_archive_orders.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       6   (1024, 2048)   int16   
  1                1 BinTableHDU     31   2R x 5C   [8A, E, E, 75D, 2J]   


In [8]:
tbl = Table.read(fits_file)

In [9]:
tbl

FUNC,XMIN,XMAX,"COEFF [15,5]",DIMS [2]
bytes8,float32,float32,float64,int32
legendre,0.0,2047.0,-29.488265991210938 .. 0.056760191917419434,1024 .. 2048
legendre,0.0,2047.0,16.73930549621582 .. 0.06081448867917061,1024 .. 2048


### Save file

In [16]:
# This is the file called by mage_make_wvguess.pro
sav_file = os.path.join(mase_path, 'MagE_wvguess_jfh.idl')

In [17]:
mase_dict = readsav(sav_file)

In [18]:
mase_dict.keys()

dict_keys(['all_arcfit', 'guess_ordr', 'rejstr', 'sv_aspec', 'sv_lines'])

In [42]:
mase_dict['sv_aspec'].shape

(50, 2048)

In [45]:
mase_dict['sv_aspec'][0]

array([ 0.      ,  0.      ,  0.      , ...,  9.345445, 15.956247,
       16.58851 ], dtype=float32)

In [22]:
mase_sol = Table(mase_dict['all_arcfit'])
mase_sol[0:15]

FUNC,NORD,NRM,LSIG,HSIG,NITER,MINPT,MAXREJ,FLG_REJ,RMS,FFIT
object,int32,object,float32,float32,int32,int32,int32,int16,float64,object
CHEBY,4,[1438.34936523 1156.48046875],2.0,2.0,3,2,10,1,0.08395581692457199,[ 3.17522658e+03 1.30876111e+02 -1.63917588e+00 -2.28740563e-01]
CHEBY,4,[1264.26269531 1347.25488281],2.0,2.0,3,1,10,1,0.029286876320838928,[ 3.30072211e+03 1.62655519e+02 -2.09591494e+00 -2.50019866e-01]
CHEBY,4,[1262.87695312 1397.08422852],2.0,2.0,3,5,10,1,0.02182971127331257,[ 3.48423411e+03 1.77976047e+02 -2.32882913e+00 -2.89890825e-01]
CHEBY,5,[1230.67089844 1494.09765625],2.0,2.0,3,5,10,1,0.01800452172756195,[ 3.68080503e+03 2.01823819e+02 -2.66847564e+00 -3.76064860e-01 5.99505038e-02]
CHEBY,5,[1069.72802734 1884.94763184],2.0,2.0,3,5,10,1,0.017211703583598137,[ 3.86377879e+03 2.72407149e+02 -3.75086778e+00 -8.11355398e-01 1.85497801e-02]
CHEBY,6,[1032.24987793 1968.95336914],2.0,2.0,3,5,10,1,0.018749188631772995,[ 4.11007866e+03 3.03810572e+02 -4.13299244e+00 -9.49344614e-01 2.05792848e-02 3.73163644e-02]
CHEBY,5,[1119.57739258 1812.52099609],2.0,2.0,3,5,10,1,0.020683664828538895,[ 4.43342265e+03 2.98381705e+02 -4.18316990e+00 -8.15175520e-01 4.10919135e-02]
CHEBY,6,[1086.2557373 1877.31591797],2.0,2.0,3,5,10,1,0.030145803466439247,[ 4.76285002e+03 3.33158942e+02 -4.62836845e+00 -9.29628940e-01 3.98870604e-02 3.21320970e-02]
CHEBY,6,[1047.34692383 1954.54882812],2.0,2.0,3,2,10,1,0.033905938267707825,[ 5.14494175e+03 3.76122825e+02 -5.13063093e+00 -1.15591072e+00 2.96200137e-02 2.64080046e-02]


#### Test one

In [28]:
order = 0
wv_air = templates.cheby_val(mase_sol['FFIT'][order], np.arange(2048), mase_sol['NRM'][order], mase_sol['NORD'][order])

In [29]:
wv_air

array([2843.40730216, 2843.63366852, 2843.86005608, ..., 3310.22710183,
       3310.43748426, 3310.64782984])

In [31]:
order = 14
wv_air = templates.cheby_val(mase_sol['FFIT'][order], np.arange(2048), mase_sol['NRM'][order], mase_sol['NORD'][order])

In [32]:
wv_air

array([ 9481.8261232 ,  9482.60903289,  9483.39197296, ...,
       11045.78014217, 11046.48662502, 11047.19302061])

## Build the archive -- in templates.py

### Check how we have X-Shooter
    Ugly JSON approach.  Dumb

In [34]:
xshoot_file = os.path.join(resource_filename('pypeit', 'data'), 'arc_lines', 'reid_arxiv', 'vlt_xshooter_vis1x1.json')


In [37]:
xdict = ltu.loadjson(xshoot_file)

In [41]:
xdict.keys()

dict_keys(['0', '1', '10', '11', '12', '13', '14', '2', '3', '4', '5', '6', '7', '8', '9', 'fit2d', 'par', 'steps'])

In [40]:
xdict['0'].keys()

dict_keys(['cen_disp', 'cen_wave', 'fitc', 'fmax', 'fmin', 'function', 'ions', 'mask', 'nrej', 'nspec', 'pixel_fit', 'rms', 'shift', 'spec', 'tcent', 'wave_fit', 'wave_soln', 'weights', 'xrej', 'yrej'])

### Will write a 2D array in 'wave' and 'flux'

In [46]:
new_mage_file = os.path.join(resource_filename('pypeit', 'data'), 'arc_lines', 'reid_arxiv', 'magellan_mage.fits')
mage_reid = Table.read(new_mage_file)

In [48]:
mage_reid['wave'].shape

(2048, 15)